In [1]:
import email
from email import policy
from email.parser import BytesParser
import re
import requests

# Function to parse email headers and extract relevant information
def parse_email_headers(raw_email):
    msg = BytesParser(policy=policy.default).parsebytes(raw_email)

    # Extract basic header information
    sender = msg.get('From')
    recipient = msg.get('To')
    subject = msg.get('Subject')
    date = msg.get('Date')

    # Extract IP addresses from headers (sender's and originating servers)
    sender_ip = extract_ip_from_header(msg.get_all('Received'))
    originating_ip = extract_ip_from_header(msg.get_all('Received-SPF'))

    return {
        "Sender": sender,
        "Recipient": recipient,
        "Subject": subject,
        "Date": date,
        "Sender IP": sender_ip,
        "Originating IP": originating_ip
    }

# Function to extract IP address from email headers using regex
def extract_ip_from_header(headers):
    if headers:
        for header in headers:
            ip_pattern = r'\b(\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})\b'
            match = re.search(ip_pattern, header)
            if match:
                return match.group(1)
    return None

# Function to query AbuseIPDB for IP reputation information
def query_abuse_ipdb(ip_address):
    api_key = 'your_abuseipdb_api_key'
    url = f'https://api.abuseipdb.com/api/v2/check?ipAddress={ip_address}&maxAgeInDays=90'
    headers = {
        'Key': api_key,
        'Accept': 'application/json'
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        if data['data']:
            return data['data']
    return None

# Example usage
raw_email = b'''\
From: sender@example.com
To: recipient@example.com
Subject: Test Email
Date: Tue, 24 Jul 2024 13:30:00 +0000
Received: from mail.example.com (mail.example.com [192.168.1.1]) by mailserver.example.com with ESMTPS id ABC123
Received-SPF: pass (sender IP 192.168.1.1)
'''

parsed_info = parse_email_headers(raw_email)

# Display parsed email header information
print("Header: Value")
print("-------------- ")
for header, value in parsed_info.items():
    print(f"{header}: {value}")

# Query AbuseIPDB for sender's IP reputation information
if parsed_info.get('Sender IP'):
    abuse_info = query_abuse_ipdb(parsed_info['Sender IP'])
    if abuse_info:
        print("\nAbuse reports found for sender's IP:")
        for report in abuse_info:
            print(report['categoryDescription'])
    else:
        print("\nNo abuse reports found for sender's IP.")



Header: Value
-------------- 
Sender: sender@example.com
Recipient: recipient@example.com
Subject: Test Email
Date: Wed, 24 Jul 2024 13:30:00 +0000
Sender IP: 192.168.1.1
Originating IP: 192.168.1.1

No abuse reports found for sender's IP.
